# Langchain, Pinecone and RAG

In [ ]:
# For this notebook, install the following 
!pip install python-dotenv
!pip install pinecone-client 
!pip install langchain
!pip install openai

In [ ]:
# !pip install -U langchain-openai
# !pip install neo4j

In [ ]:
# Import libraries 
# these are used throughput the notebook
import os
from dotenv import load_dotenv

## Write data into pinecone 

In [ ]:
# Install libraries 
from pinecone import Pinecone
import time
import requests
import json
import numpy as np

In [ ]:
# Here we load data into Pinecone

# Step 1: Load environment variables
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_KEY")
ai8_api_key = os.getenv("AI8_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")


# Step 2: Check what exists in the database
# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)

index = pc.Index(index_name)

time.sleep(1) # wait a moment for connection

print(index.describe_index_stats())
print("Successfully connected to the index")


# Step 3: Get the embeddings
def get_embeddings(input_text):
    url = "https://llm.api.ai8.io/query_llm"
    data = {
        'model':"text-embedding-ada-002",
        'input':input_text,
        'encoding_format':"float"
    }
    headers = {'Authorization': ai8_api_key}
    response = requests.post(url, json=data, headers=headers)

    response_data = json.loads(response.content)["data"]
    # Extract embeddings from the response
    embeddings = [data['embedding'] for data in response_data]
    return np.array(embeddings)


# Step 4: Read and prepare data
file_path = "supply_chain_textual_representations.txt"
with open(file_path, 'r') as file:
    lines = file.readlines()


# Step 5: Generate embeddings and import into Pinecone
for idx, line in enumerate(lines):
    print(f"Processing line {idx + 1}/{len(lines)}")
    # Generate embedding using AI8
    embedding = get_embeddings([line.strip()])  

    if embedding is not None and embedding.size > 0:
        # Normalize the embedding to unit length (optional)
        embedding = embedding / np.linalg.norm(embedding, axis=1).reshape(-1, 1)

        # Prepare data for insertion into Pinecone
        vector_id = str(idx)  # Using line index as a unique identifier
        # Include metadata; in this case, the original line text
        data = [(vector_id, embedding.flatten().tolist(), {"text": line.strip()})]

        # Insert the data into Pinecone
        index.upsert(vectors=data)
        # print(f"Inserted line {idx + 1} into Pinecone index.")
    else:
        print(f"Failed to process line {idx + 1}.")

print("All lines processed and inserted into Pinecone.")


In [ ]:
# Step 6: Query the Pinecone database
# Query text
query_text = "Who does Natoora supply to and who supplies to them"
print(f"The query asked: {query_text}")

# Generate the query embedding
query_embedding = get_embeddings([query_text])

if query_embedding is not None and query_embedding.size > 0:
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1).reshape(-1, 1)
    query_vector = query_embedding.flatten().tolist()

    res = index.query(vector=query_vector, top_k=10, include_metadata=True)
    for match in res.get("matches", []):
        score = match.get("score")
        vector_id = match.get("id")
        metadata = match.get("metadata", {})
        text = metadata.get("text", "No metadata text available")
        print(f"Score: {score:.2f}, ID: {vector_id}, Text: {text}")


At this stage, we've confirmed that the pinecone database is functioning correctly. Now, we'll proceed to utilize the retrievals from the pinecone database for our RAG implementation.

## First RAG Attempt

In [ ]:
# Step 1: Define the Query
rag_query_text = "Who are Lina Store's suppliers?"  

In [ ]:
# Step 2: Retrieve Embeddings for the Query
query_embedding = get_embeddings([rag_query_text])

if query_embedding is not None and query_embedding.size > 0:
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1).reshape(-1, 1)
    query_vector = query_embedding.flatten().tolist()

# Print statement for progress
print(query_vector[0])

In [ ]:
# Step 3: Retrieve Relevant Documents from Pinecone
top_k = 10
res = index.query(vector=query_vector, top_k=top_k, include_metadata=True)

In [ ]:
# Step 4: Prepare Augmented Query
# This step gets a list of the retrived text 
# Extract texts from the matches
contexts = [match["metadata"]["text"] for match in res.get("matches", [])]

# Combine retrieved contexts with the original query
augmented_query = "\n\n---\n\n".join(contexts) + "\n\n---\n\n" + rag_query_text

print(augmented_query)

In [ ]:
# Step 5: Generate Response 

# First let's create a system message to prime the model
primer = f"""You are Q&A bot designed to answer questions about restaurant supply chains. 
You are highly intelligent and you answer user questions based on the information provided 
by the user above each question. If the information can not be found in the information
provided by the user you truthfully say "I don't know". Please note that 'T2_Suppliers' 
refers to Tier 2 Suppliers, indicating they are a restaurant's suppliers' suppliers. 
This shows we are going further down the supply chain. 
Keep this in mind when formulating your response.
"""

def extract_message_oai(response_data):
    message_content = response_data.get("choices", [])[0].get("message", {}).get("content", "")
    # format the extracted message as markdown
    markdown_content = "---\n\n" + message_content + "\n\n---"
    return markdown_content

def generate_response_with_rag(augmented_text):
    model = "gpt-4"
    url = "https://llm.api.ai8.io/query_llm" 
    data = {
        "model": model, 
        "messages": [
            {"role": "system", "content": primer},
            {"role": "user", "content": augmented_text}
        ]
    }
    headers = {'Authorization': ai8_api_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        response_data = json.loads(response.content)
        model_response = extract_message_oai(response_data)
        return model_response
    else:
        return {"statusCode": response.status_code, "body": response.content}


In this primer, we've introduced Guardrails. These are instructions for the LLM model to follow when it doesn't have the necessary information to answer a question, which helps prevent the model from generating incorrect responses and avoids the occurrence of hallucinations.

In [ ]:
# Step 6: Display or Use the Generated Response
generated_response = generate_response_with_rag(augmented_query)
print(generated_response)

This was the final response from our initial RAG attempt. Now, let's compare this with outputs generated without using a RAG approach. We'll do this in two ways:

1. We'll exclude the "retrieval" step, meaning the LLM won't receive any extra information in the prompt.
2. We'll send the prompt to the LLM without providing a detailed primer, simulating what the answer would be like if we used a platform like 'ChatGPT', for example.

In [ ]:
# This is a non-Augmented Query, we pass in the original query as the input
non_augmented_response = generate_response_with_rag(rag_query_text)
print(non_augmented_response)

In [ ]:
# Define the get_resp_oai but this time with a more simple primer. 
def get_resp_oai(input_text, model):
    url = "https://llm.api.ai8.io/query_llm"
    data = {
        # Specify the model that you want to use
        "model": model,
        "messages": [
                    {"role": "system", "content": "You are a highly intelligent Q&A bot designed to answer questions about restaurant supply chains."},
                    {"role": "user", "content": input_text}
        ]
    }
    headers = {'Authorization': ai8_api_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        response_data = json.loads(response.content)
        model_response = extract_message_oai(response_data)
        return model_response
    else:
        return {"statusCode": response.status_code, "body": response.content}

non_augmented_response_2 = get_resp_oai(rag_query_text, "gpt-4")
print(non_augmented_response_2)

## RAG approach with LangChain Framework

Now that we have successfully executed the first RAG approach, the next step is to incorporate an LLM framework - specifically LangChain. This offers two significant benefits:

1. It provides a technical advantage by providing a standard interface for many use cases, and generally makes using LLMs more simple and efficient. 
2. It offers an opportunity to explore and learn various functionalities of LangChain.

Research into LangChain has revealed six key features: Prompts, LLMs, Indexes, Memory, Chains and Agents. This section will aim to explore as many of these features as possible in an exploratory and iterative learning manner before consolidating the understanding at the end in a final RAG approach for this supply chain use case. 

### Prompts

In [ ]:
# Import libraries 
from langchain import PromptTemplate

In [ ]:
# We are going to first get familiar with works with PromptTemplates 
# Then we can draw on some more functionality that LangChain offers 

# Step 1: Environment variables 
load_dotenv()
pinecone_api_key = os.getenv("PINECONE_KEY")
ai8_api_key = os.getenv("AI8_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")
openai_api_key = os.getenv("OPENAI_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)

In [ ]:
# We need the get_embeddings function created at the beginning of the notebook 
# Run that to continue 

In [ ]:
# Step 2: Augmented query function
def generate_augmented_prompt(prompt, top_k=20):
    # Step a: Retrieve embeddings for the question
    query_embedding = get_embeddings([prompt])
    
    if query_embedding is not None and query_embedding.size > 0:
        query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1).reshape(-1, 1)
        query_vector = query_embedding.flatten().tolist()
    
    # Step b: Retrieve relevant documents from Pinecone
    res = index.query(vector=query_vector, top_k=top_k, include_metadata=True)
    
    # Extract texts from the matches
    contexts = [match["metadata"]["text"] for match in res.get("matches", [])]
    
    # Step c: Prepare augmented query
    # Combine retrieved contexts with the original question
    augmented_prompt = "\n\n---\n\n".join(contexts) + "\n\n---\n\n" + prompt
    
    return augmented_prompt

In [ ]:
# Define a simple prompt template 
template = """Question: Tell me about the suppliers and tier 2 suppliers of {restaurant}."""

# Generate the prompt based on the template 
prompt = PromptTemplate(template=template, input_variables=["restaurant"])

# Get the user to enter a restaurant name
user_input_restaurant = input("Enter the name of the restaurant: ")

# Use the user input in formatting the prompt
formatted_prompt = prompt.format(restaurant=user_input_restaurant)

# Say we want to keep a consistent restaurant, comment out line below 
# formatted_prompt = prompt.format(restaurant="The Chiltern Firehouse")

# This gets the final prompt that includes all the extra useful information from the pinecone database 
augmented_text = generate_augmented_prompt(formatted_prompt, top_k=10)

# Finally we call the llm model with the prompt including the question and all relevant info 
response = generate_response_with_rag(augmented_text)
print(response)

While the code blocks above utilized Prompt Templates from LangChain, below we'll streamline the process further by harnessing more of LangChain's capabilities. This includes directly interfacing with an LLM, ensuring consistency with langchain.schema, and constructing a RAG pipeline that chains together different components and operations, demonstrating the flexibility and power of combining various APIs to achieve our outcome. 

In [ ]:
# Import libraries 
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

In [ ]:
# Rag pipeline 

# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)

# Initialize OpenAI Embeddings Model
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

vectorstore = PineconeVectorStore(
    index,
    embed_model,
    "text"
)

# Define the retriever with 'k=10' to retrieve top 10 similar documents
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Define the Large Language Model (LLM) with OpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7, openai_api_key=openai_api_key)

# Define the prompt template for generating answers based on retrieved context
template = """You are a highly intelligent Q&A bot designed to answer questions about restaurant supply chains.
Use the following pieces of retrieved context to answer the question.
If the information cannot be found in the information provided, truthfully say "I don't know".
Please note that 'T2_Suppliers' refers to Tier 2 Suppliers, indicating they are a restaurant's suppliers' suppliers.
No pre-amble in the answer.
Question: {question}
Context: {context}
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

rag_pipeline = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


query = "What restaurants are supplied by 'Hg Walter'?"

answers = []
contexts = []

answers.append(rag_pipeline.invoke(query))
contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

### LLMs

In [ ]:
# Import libraries
from langchain_openai import ChatOpenAI

In [ ]:
# Here we just explore using llm's with a direct OpenAI API key
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openai_api_key)

# Let's just test 
llm.invoke("How can I use docker to containerise my RAG API? Answer in bullet point steps")

### Chains

In [ ]:
# Import libraries
from langchain.chains import LLMChain

Before working with LangChain's LLMChain function, this section will initially delve into the underlying logic through a manual prompt engineering approach.

In this approach, we'll first identify direct suppliers for a given entity and then proceed to explore their second-tier suppliers individually. 

This sequential exploration intuitively allows for a more focused and granular understanding of the supplier network. By breaking down the analysis into smaller steps, we can potentially achieve more accurate and comprehensive outputs. Each augmented prompt, created with data from the Pinecone database, is tailored to address smaller, more focused questions. This approach could avoid dealing with too much information at once and instead and capture specific details more effectively. 

In [ ]:
# Prompt Engineering

# Template for generating a prompt to identify direct suppliers for a given entity
supplier_template = """Please determine and compile a list of all direct suppliers for {entity}, using the detailed supplier information provided. 
Focus exclusively on direct suppliers, which are specifically designated as T2_Suppliers. Direct suppliers are entities that directly provide goods or services to {entity}. 
If no T2_Supplier is found to directly supply to {entity}, the response should be left blank.

For clarity, if you are asked to identify direct suppliers of Fresh Direct and you encounter information stating, 'A T2_Supplier named Sun Salads directly supplies Watercress in Dorset, UK, to a Supplier named Fresh Direct,' 
your task is to recognize 'Sun Salads' as the direct supplier to 'Fresh Direct'. Any other information is irrelevant and should not be included in your answer."""

def prompt_chain_function(template, entity, top_k):
    # Generate prompt with the given template and entity
    prompt = PromptTemplate(template=template, input_variables=["entity"])
    formatted_prompt = prompt.format(entity=entity)
    # Augment the prompt text
    augmented_text = generate_augmented_prompt(formatted_prompt, top_k=top_k)
    # Generate response using RAG
    response = generate_response_with_rag(augmented_text)
    return response 

def clean_response_function(response):
    # Template for cleaning response
    template = """Extract the names of suppliers from this text:
    {text}
    The output should strictly consist of a comma-separated list of the direct suppliers' names, without any additional text or explanations.
    
    For example, the output should look like this: 'Supplier_1, Supplier_2, ..., Supplier_n'.
    Please ensure the list is formatted exactly as shown in the example, with each supplier's name separated by a comma and a space. 
    If there is no information, just write 'unknown'."""
    
    # Generate prompt with the given response
    prompt = PromptTemplate(template=template, input_variables=["text"])
    formatted_prompt = prompt.format(text=response)
    # Generate cleaned response using RAG
    cleaned_response = generate_response_with_rag(formatted_prompt)
    return cleaned_response

def process_supplier_info(info):
    # Removing unwanted parts from the string
    cleaned_info = info.replace('---\n\n', '').replace('\n\n---', '').strip()
    # Additional processing logic can be added here
    return cleaned_info

### Step 1: Get the suppliers of the restaurant and clean the output 
restaurant_template = """Question: Tell me about the direct suppliers {entity}."""  
user_input_restaurant = input("Enter the name of the restaurant: ")
restaurant_suppliers = prompt_chain_function(restaurant_template, user_input_restaurant, 10)
clean_restaurant_suppliers = clean_response_function(restaurant_suppliers)
clean_restaurant_suppliers = process_supplier_info(clean_restaurant_suppliers)

# Print original and cleaned responses
print(restaurant_suppliers)
print(f"{user_input_restaurant}:\n{clean_restaurant_suppliers}")
print("\n")

### Step 2: Get the suppliers of the restaurant suppliers and clean the output 
suppliers_list = clean_restaurant_suppliers.split(', ')
suppliers_list = [supplier.replace('---\n\n', '').replace('\n\n---', '') for supplier in suppliers_list]
t2_supplier_outputs = {}
for supplier in suppliers_list:
    # Generate prompt to identify T2 suppliers for each restaurant supplier
    output = prompt_chain_function(supplier_template, supplier, 20)
    t2_supplier_outputs[supplier] = output

# Process and clean T2 supplier outputs
for key, value in t2_supplier_outputs.items():
    processed_output = process_supplier_info(value)
    clean_t2_suppliers = clean_response_function(processed_output)
    final_output = process_supplier_info(clean_t2_suppliers)
    
    # Print T2 supplier outputs
    print(f"{key}'s suppliers: \n{final_output}\n")

Now that we can confirm this has worked, and from manual review, the answer is accurate and includes all relevant information, the next stage is to see if we can replicate the idea of where one answer feeds into another, but we will LangChain's LLMChain function. 

In [ ]:
# We use the generate_augmented_prompt function defined above 
# Run that function before continuing 

In [ ]:
# First, we are going to use LLMChains
# This allows  for the creation of sequences (chains) where the output of one model can serve as the input for another, 
# enabling complex, multistep information processing workflows.
# This is akin to prompt engineering 

# Step 1: Define the templates
template_level1 = """"Identify and list all direct suppliers of {entity}, based on the provided supplier information. 
It's important to note that we are specifically looking for direct suppliers only, not Tier 2 Suppliers (also known as T2 Suppliers). 
The output should strictly consist of a comma-separated list of the direct suppliers' names, without any additional text or explanations.

For example, the output should look like this: 'Supplier_1, Supplier_2, ..., Supplier_n'.
Please ensure the list is formatted exactly as shown in the example, with each supplier's name separated by a comma and a space. 

Use the following supplier information to support your answer:
{supplier_info}
"""

template_level2 = """"Please determine and compile a list of all direct suppliers for {entity}, using the detailed supplier information provided. 
Focus exclusively on direct suppliers, which are specifically designated as T2_Suppliers. Direct suppliers are entities that directly provide goods or services to {entity}. 
If no T2_Supplier is found to directly supply {entity}, the response should be left blank.

For clarity, if you are asked to identify direct suppliers of Fresh Direct and you encounter information stating, 'A T2_Supplier named Sun Salads directly supplies Watercress in Dorset, UK, to a Supplier named Fresh Direct,' 
your task is to recognize 'Sun Salads' as the direct supplier to 'Fresh Direct'. Any other information is irrelevant and should not be included in your answer. 

Your response must be formatted as a comma-separated list containing the names of these direct suppliers, without including any additional text or explanations. 

Ensure your output adheres to the following format: 'Supplier_1, Supplier_2, ..., Supplier_n'. Each supplier's name should be clearly delineated, separated by a comma followed by a space. 
Use the following supplier information to support your answer:
{supplier_info}
"""

In [ ]:
# Step 2: Create the LLM to use 
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=openai_api_key)

In [ ]:
# Step 3: Generate the functions 
def generate_t1_suppliers(template, top_k, user_input_restaurant):
    template = template 
    rag_template = """List all of the suppliers of {entity}."""
    rag_prompt = PromptTemplate(input_variables=["entity"], template=rag_template) 

    formatted_prompt = rag_prompt.format(entity=user_input_restaurant)
    restaurant_suppliers = generate_augmented_prompt(formatted_prompt, top_k=top_k)

    # Now we call the llm with the query 
    prompt = PromptTemplate(input_variables=["entity", "supplier_info"], template=template) 
    suppliers_output = LLMChain(llm=llm, prompt=prompt, output_key='suppliers')
    t1_output = suppliers_output.run(entity=user_input_restaurant, supplier_info=restaurant_suppliers)
    
    return t1_output


# The function for tier 2 suppliers
def generate_t2_suppliers(template, top_k, output=None):
    template = template 
    rag_template = """List all of the T2_Suppliers of {entity}."""
    rag_prompt = PromptTemplate(input_variables=["entity"], template=rag_template) 
    # print("Going level 2 route")
    suppliers_list = output.split(', ')
    t2_supplier_outputs = {}

    for supplier in suppliers_list:
        # print(supplier)
        formatted_prompt = rag_prompt.format(entity=supplier)
        supplier_suppliers = generate_augmented_prompt(formatted_prompt, top_k=top_k)

        # Now we call the llm with the query 
        prompt = PromptTemplate(input_variables=["entity", "supplier_info"], template=template) 
        suppliers_output = LLMChain(llm=llm, prompt=prompt)

        t2_output = suppliers_output.run(entity=supplier, supplier_info=supplier_suppliers)
        t2_supplier_outputs[supplier] = t2_output

    return t2_supplier_outputs

In [ ]:
# Run the functions
user_input_restaurant = input("Enter the name of the restaurant: ")
print("\n")

output = generate_t1_suppliers(template=template_level1, top_k=10, user_input_restaurant=user_input_restaurant)

print(f"{user_input_restaurant}'s Tier 1 Suppliers are: {output}")
print("\n")

t2_supplier_outputs = generate_t2_suppliers(template=template_level1, top_k=10, output=output)

print(t2_supplier_outputs)

### Memory

This next section is an example of creating a retrieval-based conversational agent that can handle detailed inquiries by leveraging both document retrieval mechanisms and conversational context. It showcases how chains and memory (in the form of conversational history) are utilized together to create more sophisticated and context-aware responses.

So, 'chat_history' is used to store and reference the conversation history as part of the input for the conversational retrival and the code also utilised multiple different types of chains such as 'create_stuff_documents_chain', 'create_retrieval_chain', 'create_history_aware_retriever'.

This represents another use case and opportunity to learn how LangChain can be used for this an LLM solution.

In [ ]:
# Import libraries

# a) Document chain creation
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# b) Retrieval chain creation
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_retrieval_chain

# c) Conversational Retrieval Chain 
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# d) Example Invocation and Conversational Context Management
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
# Step 1. Document chain creation 
# This section initializes a chain for processing documents. It involves creating a template for querying and instantiating a document chain.

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
# Step 2. Retrieval Chain Setup
# # Setup for a chain dedicated to retrieving documents based on a given query. This includes initializing a retriever and creating a retrieval chain that utilizes the previously defined document chain.

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)

# Initialize OpenAI Embeddings Model
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

# Set up the vectorstore 
vectorstore = PineconeVectorStore(
    index,
    embed_model,
    "text"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Testing it works
response = retrieval_chain.invoke({
    "input": "Who supplies Lina Stores?"
})
lina_stores_answer = response['answer'] 
print(lina_stores_answer)

In [ ]:
# Step 3. Conversational Retrieval Chain Initialization
# Creating a chain that is aware of the conversation history for more contextual retrieval. This involves setting up templates and chains that take into account the chat history.

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
# Step 4: Example Invocation and Conversational Context Management
# This section demonstrates how to invoke the conversational retrieval chain with a specific query and manage the chat history to simulate a dynamic conversation.

chat_history = [
    HumanMessage(content="Who supplies Lina Stores?"),
    AIMessage(content=lina_stores_answer)
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# Manually test 
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "Who supplies Lina Stores?"
})

message_answer = response['answer']

print(f"The response output, including 'chat_history', 'input', 'context' and 'message':\n {response}\n")

print(f"Only the response 'message' output:\n {message_answer}")

# After first invocation
chat_history.append(HumanMessage(content="Tell me more about it!"))
chat_history.append(AIMessage(content=message_answer))  # Assuming you add the actual response

# Now the chat_history includes the new exchange, and we can invoke the chain again with updated context
response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Can you find any other restaurants supplied by those suppliers"
})

answer = response['answer']
print(f"Printing the next answer in the conversation:\n {answer}")

In [ ]:
# Step 5: Conversational Loop (Interactive Querying)
# Interactive loop allowing users to ask questions in a conversational context. This section demonstrates how the system can handle live user input and respond accordingly.

# Given that the conversational_retrieval_chain is already defined and initialized

# Define the function to ask a question, reference chat history and retrieval_chain (from Pinecone vector database), print the answer and update chat history

def ask_question(input_question):
    global chat_history  # Reference the global chat history

    # Invoke the conversational retrieval chain with the current chat history and the input question
    response = conversational_retrieval_chain.invoke({
        'chat_history': chat_history,
        "input": input_question
    })

    answer = response['answer']
    print(f"AI: {answer}")

    # Update chat history with the new exchange
    chat_history.append(HumanMessage(content=input_question))
    chat_history.append(AIMessage(content=answer))

# Interactive loop for asking questions
while True:
    user_input = input("Ask a question: ")  # Get input question from the user
    if user_input.lower() == "quit":  # Define a way to exit the loop
        print("Exiting.")
        break
    ask_question(user_input)
    

### Indexes 
* Document loaders 
* Vector databases
* Text splitters

In [ ]:
# Import libraries

# a) vector database 
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# b) document loaders 
from langchain_community.document_loaders import WebBaseLoader

# c) text splitters 
from langchain.text_splitter  import RecursiveCharacterTextSplitter

#### (a) Vector Database

In [ ]:
# Initialize Pinecone
pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_name)

# Initialize OpenAI Embeddings Model
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=openai_api_key)

# Set up the vectorstore 
vectorstore = PineconeVectorStore(
    index,
    embed_model,
    "text"
)

# Example query
query = "Who supplies Lina Stores??"

# Test functionality to retrieve data from Pinecone directly using LangChain
vectorstore.similarity_search(query, k=5)

#### (b) Document Loaders

In [ ]:
# Additional information to write up a sustainability report 
# This is for a more comprehensive example use case 

loader = WebBaseLoader("https://www.sustainableagriculture.eco/post/navigating-the-complexity-of-sustainable-food-supply-chains")

docs = loader.load()
print(docs)

#### (c) Text Splitters

In [ ]:
# Here let's split the docs up into chunks 
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# print statement for progress
print(documents)

#### Example use case 

In [ ]:
# Load libraries 
from tqdm.auto import tqdm  # For progress indication

In [ ]:
# First we have to add all the data from documents into the Pinecone database 

# Step 1: Generate embeddings for your documents
# Split documents might have resulted in a list of strings (documents)
# Let's generate embeddings for these documents
embeddings_list = []
for doc in tqdm(documents):
    embedding = embed_model.embed_text(doc)  # Ensure this method matches your actual embedding method
    embeddings_list.append(embedding)

# Step 2: Prepare data for Pinecone
# Creating a unique identifier for each document 
data_to_upsert = []
for idx, (doc, emb) in enumerate(zip(documents, embeddings_list)):
    unique_id = f"doc_{idx}"  # You might want to create more sophisticated IDs based on your documents
    data_to_upsert.append((unique_id, emb, {"text": doc}))

# Step 3: Upsert data into Pinecone
# This can be done in batches to avoid overloading memory with large datasets
batch_size = 100 
for i in tqdm(range(0, len(data_to_upsert), batch_size)):
    batch = data_to_upsert[i:i+batch_size]
    index.upsert(vectors=batch)


In [ ]:
### Here we want to use the output so the LLM can act as sustainability officer for a restaurant 
# The final text is a sustainability paragraph on behalf of the restaurant 

# First let's create a system message to prime the model
primer = f"""
You are a sustainability officer for a restaurant, tasked with understanding and optimizing the supply chain for both environmental sustainability and human rights. 
Your role involves assessing the sources of ingredients used in the restaurant, focusing on the transparency and sustainability of these sources.

If comprehensive details of the supply chain are available, utilize this information to highlight sustainable practices or identify areas for improvement. 
In cases where information about suppliers or their practices is incomplete, note the need to seek out this information to enhance supply chain transparency.

Your approach should include:
- Identifying known suppliers and their contributions to the restaurant's supply chain.
- Acknowledging gaps in the supply chain information, especially regarding secondary suppliers or the origins of specific ingredients.
- Emphasizing the importance of continuous improvement and investigation to ensure the sustainability and ethical standards of the supply chain are met.
- Write 1-2 Paragraphs that can be included in the restaurant's annual sustainability report.
"""

# The function generate_response_with_rag needs to be ran before continuing 

# Combining the information
# This info is taken from the prompt engineering approach 
combined_info = restaurant_suppliers + "\n\nTier 2 Supplier Information:\n"

for supplier, details in t2_supplier_outputs.items():
    combined_info += f"\n{supplier}:\n{details}"

generate_response_with_rag(combined_info)

#### Agents 
[insert text here]

In [ ]:
# Import libraries 